# **Import Libraries**

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:

def detect_ball_color(frame, ball_type, table_color='green'): 
    """
    Detect ball using color segmentation with table background removal
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # The Colour of ball
    color_ranges = {
        'white': ([0, 0, 200], [180, 30, 255]),
        'blue': ([100, 150, 50], [130, 255, 255]),
        'black': ([0, 0, 0], [180, 255, 50]),
        'red': ([0, 120, 70], [10, 255, 255]),
        'yellow': ([20, 100, 100], [30, 255, 255]),
        'pink': ([140, 100, 100], [170, 255, 255]),
        'green': ([35, 40, 40], [85, 255, 255])  
    }
    
    # COlour of table 
    table_ranges = {
        'green': ([40, 50, 50], [80, 255, 255]),   # For Green table
        'blue': ([90, 50, 50], [130, 255, 255]),   # For Blue table
        'red': ([0, 50, 50], [10, 255, 255])       # For Red table
    }
    
    # Ball masking
    lower, upper = color_ranges.get(ball_type, ([0, 0, 0], [180, 255, 255]))
    ball_mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    
    # Table masking
    if table_color in table_ranges:
        table_lower, table_upper = table_ranges[table_color]
        table_mask = cv2.inRange(hsv, np.array(table_lower), np.array(table_upper))
        # Remove table from ball mask
        ball_mask = cv2.bitwise_and(ball_mask, cv2.bitwise_not(table_mask))
    
    # Morphology
    kernel = np.ones((5, 5), np.uint8)
    ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_CLOSE, kernel)
    ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_OPEN, kernel)
    
    return ball_mask

def track_single_ball_two_outputs(video_path, ball_type='white', table_color='green',
                                   output_with_bg='tracked_with_background.mp4',
                                   output_no_bg='tracked_black_background.mp4'):
    
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    

    MIN_BALL_RADIUS = int(height * 0.008) 
    MAX_BALL_RADIUS = int(height * 0.035) 
    MIN_BALL_AREA = int(np.pi * MIN_BALL_RADIUS ** 2)
    MAX_BALL_AREA = int(np.pi * MAX_BALL_RADIUS ** 2)
    
    print(f"Video: {width}x{height}, FPS: {fps:.1f}")
    print(f"Ball size limits: radius {MIN_BALL_RADIUS}-{MAX_BALL_RADIUS}px, area {MIN_BALL_AREA}-{MAX_BALL_AREA}px²")
    print(f"Tracking: {ball_type} ball on {table_color} table")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_with_bg = cv2.VideoWriter(output_with_bg, fourcc, fps, (width, height))
    out_no_bg = cv2.VideoWriter(output_no_bg, fourcc, fps, (width, height))
    
    trajectory = []
    frame_count = 0
    last_valid_position = None
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect ball
        mask = detect_ball_color(frame, ball_type, table_color)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ball_detected = False
        ball_position = None
        ball_radius = 0
        best_contour = None
        
        if contours:
            best_score = 0
            
            for contour in contours:
                area = cv2.contourArea(contour)
                
                if MIN_BALL_AREA < area < MAX_BALL_AREA:
                    perimeter = cv2.arcLength(contour, True)
                    if perimeter > 0:
                        circularity = 4 * np.pi * area / (perimeter * perimeter)
                        
                        if circularity > 0.7:
                            (x, y), radius = cv2.minEnclosingCircle(contour)
                            
                            if MIN_BALL_RADIUS < radius < MAX_BALL_RADIUS:
                                score = circularity * 100
                                
                                if last_valid_position is not None:
                                    dist = np.sqrt((x - last_valid_position[0])**2 + 
                                                 (y - last_valid_position[1])**2)
                                    if dist < 150:  
                                        score += 50
                                
                                if score > best_score:
                                    best_score = score
                                    ball_position = (int(x), int(y))
                                    ball_radius = int(radius)
                                    best_contour = contour
                                    ball_detected = True
        
        frame_with_tracking = frame.copy()
        
        frame_black_bg = np.zeros_like(frame)
        
        if ball_detected and ball_radius > 0:
            last_valid_position = ball_position
            
            cv2.circle(frame_with_tracking, ball_position, ball_radius, (0, 255, 255), 3)
            cv2.circle(frame_with_tracking, ball_position, 4, (0, 0, 255), -1)
            
            trajectory.append(ball_position)
            if len(trajectory) > 50:  
                trajectory.pop(0)
            
            for i in range(1, len(trajectory)):
                if trajectory[i-1] is not None and trajectory[i] is not None:
                    thickness = int(2 + (i / len(trajectory)) * 2)
                    cv2.line(frame_with_tracking, trajectory[i-1], trajectory[i], 
                            (255, 255, 0), thickness)
            
            cv2.putText(frame_with_tracking, f"{ball_type.upper()} BALL", 
                       (ball_position[0] - 40, ball_position[1] - ball_radius - 15),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            
            mask_ball = np.zeros((height, width), dtype=np.uint8)
            cv2.circle(mask_ball, ball_position, ball_radius, 255, -1)
            
            for c in range(3):
                frame_black_bg[:, :, c] = cv2.bitwise_and(frame[:, :, c], 
                                                          frame[:, :, c], 
                                                          mask=mask_ball)
        
        out_with_bg.write(frame_with_tracking)
        out_no_bg.write(frame_black_bg)
        
        display_scale = 0.6
        display_with = cv2.resize(frame_with_tracking, None, fx=display_scale, fy=display_scale)
        display_black = cv2.resize(frame_black_bg, None, fx=display_scale, fy=display_scale)
        display_mask = cv2.resize(mask, None, fx=display_scale, fy=display_scale)
        
        cv2.imshow('1. With Background', display_with)
        cv2.imshow('2. Black Background', display_black)
        cv2.imshow('3. Detection Mask', display_mask)
        
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out_with_bg.release()
    out_no_bg.release()
    cv2.destroyAllWindows()
    
    print(f"\n Created two videos:")
    print(f"1. With background: {output_with_bg}")
    print(f"2. Black background: {output_no_bg}")
    print(f"Total frames: {frame_count}")

def track_multiple_balls(video_path, ball_types=['white', 'blue'], table_color='green', #change colour for multiple tracking
                        output_video='multi_ball_tracking.mp4'):
    """
    Track multiple balls simultaneously
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    MIN_BALL_RADIUS = int(height * 0.008)
    MAX_BALL_RADIUS = int(height * 0.035)
    MIN_BALL_AREA = int(np.pi * MIN_BALL_RADIUS ** 2)
    MAX_BALL_AREA = int(np.pi * MAX_BALL_RADIUS ** 2)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    
    ball_colors_display = {
        'white': (255, 255, 255),
        'blue': (255, 0, 0),
        'red': (0, 0, 255),
        'black': (50, 50, 50),
        'yellow': (0, 255, 255),
        'green': (0, 255, 0),
        'pink': (255, 0, 255)
    }
    
    trajectories = {ball_type: [] for ball_type in ball_types}
    frame_count = 0
    
    print(f"Tracking multiple balls: {ball_types} on {table_color} table")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        display_frame = frame.copy()
        
        for ball_type in ball_types:
            mask = detect_ball_color(frame, ball_type, table_color)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            best_contour = None
            best_score = 0
            
            for contour in contours:
                area = cv2.contourArea(contour)
                
                if MIN_BALL_AREA < area < MAX_BALL_AREA:
                    perimeter = cv2.arcLength(contour, True)
                    if perimeter > 0:
                        circularity = 4 * np.pi * area / (perimeter * perimeter)
                        
                        if circularity > 0.7:
                            (x, y), radius = cv2.minEnclosingCircle(contour)
                            
                            if MIN_BALL_RADIUS < radius < MAX_BALL_RADIUS:
                                score = circularity * area
                                if score > best_score:
                                    best_score = score
                                    best_contour = contour
            
            if best_contour is not None:
                (x, y), radius = cv2.minEnclosingCircle(best_contour)
                position = (int(x), int(y))
                radius = int(radius)
                
                color = ball_colors_display.get(ball_type, (0, 255, 255))
                
                cv2.circle(display_frame, position, radius, color, 3)
                cv2.circle(display_frame, position, 4, color, -1)
                
                trajectories[ball_type].append(position)
                if len(trajectories[ball_type]) > 30:
                    trajectories[ball_type].pop(0)
                
                traj = trajectories[ball_type]
                for i in range(1, len(traj)):
                    cv2.line(display_frame, traj[i-1], traj[i], color, 2)
                
                cv2.putText(display_frame, ball_type.upper(), 
                           (position[0] - 30, position[1] - radius - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        out.write(display_frame)
        cv2.imshow('Multi-Ball Tracking', cv2.resize(display_frame, None, fx=0.6, fy=0.6))
        
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"\nMulti ball tracking  {output_video}")

#  HOW TO USE 
if __name__ == "__main__":
    
    VIDEO_PATH = "./test.mp4"  #  CHANGE YOUR VIDEO PATH CTRL+ALT+C
    
    # OPTION 1: SINGLE BALL TRACKING 
    track_single_ball_two_outputs(
        video_path=VIDEO_PATH,
        ball_type='yellow',           # CHANGE: 'white', 'blue', 'black', 'red', 'yellow', 'pink', 'green'
        table_color='blue',         # CHANGE: 'green', 'blue', 'red'
        output_with_bg='tracked_with_bg.mp4',
        output_no_bg='white_ball_only.mp4'
    )
    
    

Video: 1280x720, FPS: 30.0
Ball size limits: radius 5-25px, area 78-1963px²
Tracking: yellow ball on blue table
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...
Processed 120 frames...
Processed 150 frames...
Processed 180 frames...
Processed 210 frames...
Processed 240 frames...
Processed 270 frames...
Processed 300 frames...
Processed 330 frames...
Processed 360 frames...
Processed 390 frames...
Processed 420 frames...
Processed 450 frames...
Processed 480 frames...
Processed 510 frames...
Processed 540 frames...
Processed 570 frames...
Processed 600 frames...
Processed 630 frames...
Processed 660 frames...
Processed 690 frames...
Processed 720 frames...
Processed 750 frames...
Processed 780 frames...
Processed 810 frames...
Processed 840 frames...
Processed 870 frames...
Processed 900 frames...
Processed 930 frames...
Processed 960 frames...
Processed 990 frames...
Processed 1020 frames...
Processed 1050 frames...
Processed 1080 frames...
Processed 1110 frames...

# **For Multiple Ball tracking**

In [3]:
# OPTION 2: MULTIPLE BALL TRACKING 
#      Uncomment to track multiple balls at once:
   
track_multiple_balls(
        video_path=VIDEO_PATH,
        ball_types=['white', 'red', 'yellow'],  #  CHANGE ball colors
        table_color='blue',                     # CHANGE table color
        output_video='multi_ball_tracking.mp4'
    )

Tracking multiple balls: ['white', 'red', 'yellow'] on blue table
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...
Processed 120 frames...
Processed 150 frames...
Processed 180 frames...
Processed 210 frames...
Processed 240 frames...
Processed 270 frames...
Processed 300 frames...
Processed 330 frames...
Processed 360 frames...
Processed 390 frames...
Processed 420 frames...
Processed 450 frames...
Processed 480 frames...
Processed 510 frames...
Processed 540 frames...
Processed 570 frames...
Processed 600 frames...
Processed 630 frames...
Processed 660 frames...
Processed 690 frames...
Processed 720 frames...
Processed 750 frames...
Processed 780 frames...
Processed 810 frames...
Processed 840 frames...
Processed 870 frames...
Processed 900 frames...
Processed 930 frames...
Processed 960 frames...
Processed 990 frames...
Processed 1020 frames...
Processed 1050 frames...
Processed 1080 frames...
Processed 1110 frames...
Processed 1140 frames...
Processed 1170 frame